In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path
import os
from data_io import get_book

In [2]:
metadata_filename = 'metadata.csv'
counts_dirname = 'counts'
tokens_dirname = 'tokens'

metadata_df = pd.read_csv(metadata_filename)

filtered_df = metadata_df[(metadata_df.language == "['en']") & (metadata_df.type == 'Text')]

SELECTED_COLUMNS = ['id', 'title', 'author', 'authoryearofbirth', 'authoryearofdeath']
filtered_df = filtered_df.dropna(subset=SELECTED_COLUMNS)
filtered_df = filtered_df[SELECTED_COLUMNS]
filtered_df = filtered_df.reset_index(drop=True)

author_count = filtered_df['author'].value_counts()
many_works_author = author_count[author_count >= 10]
filtered_df = filtered_df[filtered_df.author.isin(many_works_author.index.to_numpy())].reset_index()

'PG8700' in filtered_df.id

False

In [7]:
# filtered_df = filtered_df.sample(n=50, random_state=2).reset_index()

sampled_authors = filtered_df.author.sample(n=50, random_state=1)

train_ids = []
test_ids = []
val_ids = []

for author in sampled_authors:
    works = filtered_df[filtered_df.author == author].sample(n=3, random_state=1)
    train_id, test_id, val_id = works.id
    
    # Does not check if this file exists and is valid
    
    train_ids.append(train_id)
    test_ids.append(test_id)
    val_ids.append(val_id)
    

train_df = filtered_df[filtered_df.id.isin(train_ids)]
test_df = filtered_df[filtered_df.id.isin(test_ids)]
val_df = filtered_df[filtered_df.id.isin(val_ids)]

df_arrs = []
for df in [train_df, test_df, val_df]:
    df = df[['id', 'author']]

    docs = []
    docs_unavail_pg_ids = []
    for pg_id in df.id:    
        try:
            doc = ' '.join(get_book(pg_id, os.path.join(tokens_dirname), level='tokens'))
            docs.append(doc)
        except:
            docs_unavail_pg_ids.append(pg_id)
    
    df = df[~df.id.isin(docs_unavail_pg_ids)].reset_index()
    df.insert(2, 'text', docs, True)
    
    df_arrs.append(df)

train_df, test_df, val_df = df_arrs

In [8]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)
val_df.to_csv('val.csv', index=False)